# Data Augmentation


In [19]:
import os

import matplotlib.pyplot as plt
import numpy as np

import warnings
warnings.filterwarnings('ignore')
import numpy as np
import skimage.io as io
from skimage.transform import rotate, AffineTransform, warp
from skimage.util import random_noise
from skimage.filters import gaussian

from skimage.io import imread, imsave
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from skimage.transform import rotate
from skimage.util import random_noise
from skimage.filters import gaussian
from scipy import ndimage
import torch
from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from torch.utils.data import DataLoader
from torchvision import transforms




from src.datasets import SatelliteImagesDataset, train_test_split

from src.path import DATA_TRAIN_IMG_PATH, MODELS_DIR, extract_archives
from src.path import DATA_TRAIN_GT_PATH

from src.plot_utils import plot_images


from numpy import asarray

NUM_EPOCHS = 100
MODEL_PATH = os.path.join(MODELS_DIR, 'convnet_model.pth')

from PIL import Image
from PIL import ImageFilter


Load Path

In [20]:
#Path to save Augmented Image
from src.path import DATA_DIR
AUGMENTED_IMAGES_DIR = os.path.join(DATA_DIR,'augmented/images')
AUGMENTED_GT_DIR = os.path.join(DATA_DIR,'augmented/groundtruth')

In [21]:
dataset = SatelliteImagesDataset(
        img_dir=DATA_TRAIN_IMG_PATH,gt_dir=DATA_TRAIN_GT_PATH)

image, mask = dataset[0]

#plot_images(image, mask)

type(image)

PIL.PngImagePlugin.PngImageFile

# Augmentation

In [29]:
noise = np.random.randint(5, size = (400, 400, 3), dtype = 'uint8')

#Better if its aleatory generated
box = (50, 50, 300, 300)

In [30]:
#Enumerate Data
for i, data in enumerate(dataset):
    
    if i>0 :
        continue
    
    image, mask = data
    #Get the numpy array
    np_image = np.array(image)
    np_gt = np.array(mask)
    
    #---------Flip Left-Right--------
    np_im_flip = np.fliplr(np_image)
    np_gt_flip = np.fliplr(np_gt)
    img_flip = Image.fromarray(np_im_flip, 'RGB')
    gt_flip = Image.fromarray(np_gt_flip, 'L')
    img_flip.save(f"{AUGMENTED_IMAGES_DIR}/satImage_FlipLR_{i}.png")
    gt_flip.save(f"{AUGMENTED_GT_DIR}/satImage_FlipLR_{i}.png")
    
    #---------Flip Up-Down ---------
    im_flipUD= image.transpose(Image.FLIP_TOP_BOTTOM)
    gt_flipUD= mask.transpose(Image.FLIP_TOP_BOTTOM)
    im_flipUD.save(f"{AUGMENTED_IMAGES_DIR}/satImage_FlipUD_{i}.png")
    gt_flipUD.save(f"{AUGMENTED_GT_DIR}/satImage_FlipUD_{i}.png")
    
    #---------Rotate-----------------
    img_rot = image.rotate(45)
    gt_rot= mask.rotate(45)
    img_rot.save(f"{AUGMENTED_IMAGES_DIR}/satImage_Rot_{i}.png")
    gt_rot.save(f"{AUGMENTED_GT_DIR}/satImage_Rot_{i}.png")
    
    #---------Random Noise------------
    np_im_noise = np_image
    
    for f in range(400):
        for j in range(400):
            for k in range(3):
                if (np_im_noise[f][j][k] != 255):
                    np_im_noise[f][j][k] += noise[f][j][k]
    
    img_noise = Image.fromarray(np_im_noise, 'RGB')
    img_noise.save(f"{AUGMENTED_IMAGES_DIR}/satImage_Noise{i}.png")
    #No change for the mask
    mask.save(f"{AUGMENTED_GT_DIR}/satImage_Noise{i}.png")
    
    #--------Blurr----------------------
    
    img_blurr = image.filter(ImageFilter.BLUR)
    
    img_blurr.save(f"{AUGMENTED_IMAGES_DIR}/satImage_Blurr{i}.png")
    #No change for the Mask
    mask.save(f"{AUGMENTED_GT_DIR}/satImage_Blurr{i}.png")
    
    #-------CROP------------------------
    im_crop = image.crop(box)
    gt_crop = mask.crop(box)
    im_crop.save(f"{AUGMENTED_IMAGES_DIR}/satImage_Crop_{i}.png")
    gt_crop.save(f"{AUGMENTED_GT_DIR}/satImage_Crop_{i}.png")

    